In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [101]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, StratifiedShuffleSplit
from sklearn.metrics import average_precision_score, roc_auc_score
import numpy as np
from sklearn import tree
from deslib.des.knora_e import KNORAE
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
import torch
from time import time
from skopt import gp_minimize
from skopt.space import Integer, Real
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import average_precision_score
from sklearn.model_selection import RandomizedSearchCV

In [336]:
chembl = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/chembl_4_smiles.csv', index_col=0)

In [40]:
chembl.loc[chembl['smiles'].str.contains('\.')]

,chembl_id,pubchem_cid,smiles,parent_smiles,chembl_tox,withdrawn
464,CHEMBL1237066,62859,O=C([O-])C(O)[C@H](O)[C@@H](O)[C@H](O)[C@H](O)...,O=C([O-])C(O)[C@H](O)[C@@H](O)[C@H](O)[C@H](O)...,Safe,0
700,CHEMBL2010412,452192,C[n+]1c2cc(N)ccc2cc2ccc(N)cc21.Nc1ccc2cc3ccc(N...,missing,Safe,0
842,CHEMBL1200747,62358,CC(O)C(=O)O.N,CC(O)C(=O)O.N,Safe,0
1376,CHEMBL2106975,61102,O=C([O-])O.[K+],O=C([O-])O.[K+],Safe,0
1409,CHEMBL1255943,2723891,Cl.N[C@@H](CCC(=O)O)C(=O)O,Cl.N[C@@H](CCC(=O)O)C(=O)O,Safe,0
...,...,...,...,...,...,...
10391,CHEMBL1200691,8896,CC(=O)[O-].CC(=O)[O-].[Mg+2],CC(=O)[O-].CC(=O)[O-].[Mg+2],Safe,0
10524,CHEMBL2106123,13136,O=C([O-])CC(O)(CC(=O)[O-])C(=O)[O-].O=C([O-])C...,O=C([O-])CC(O)(CC(=O)[O-])C(=O)[O-].O=C([O-])C...,Safe,0
10531,CHEMBL2364968,90661668,CC(C)C[C@H]1C(=O)N2CCC[C@H]2[C@]2(O)O[C@](NC(=...,missing,Safe,0
10560,CHEMBL261772,missing,C=C1c2c(Cl)ccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(...,C=C1c2c(Cl)ccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(...,Safe,0


In [338]:
chembl['smiles'].str.unique()

AttributeError: 'StringMethods' object has no attribute 'unique'

# Models

In [125]:
search_space = { # values for boostrap can be either True or False # values of max_depth are integers from 6 to 20
        "max_iter": Integer(10, 1000),
        "learning_rate": Real(0.001, 1),  
        "min_samples_leaf": Integer(1, 30)
    }

In [119]:
forest_clf = HistGradientBoostingClassifier(early_stopping=True, validation_fraction=0.15)

In [126]:
forest_bayes_search = BayesSearchCV(forest_clf, search_space, n_iter=32, # specify how many iterations
                                    scoring="roc_auc", n_jobs=-1, cv=5)

In [127]:
forest_bayes_search.fit(X, y)

BayesSearchCV(cv=5,
              estimator=HistGradientBoostingClassifier(early_stopping=True,
                                                       validation_fraction=0.15),
              n_iter=32, n_jobs=-1, scoring='roc_auc',
              search_spaces={'learning_rate': Real(low=0.001, high=1, prior='uniform', transform='identity'),
                             'max_iter': Integer(low=10, high=1000, prior='uniform', transform='identity'),
                             'min_samples_leaf': Integer(low=1, high=30, prior='uniform', transform='identity')})

In [128]:
forest_bayes_search.best_estimator_

HistGradientBoostingClassifier(early_stopping=True,
                               learning_rate=0.01619991660275452, max_iter=25,
                               min_samples_leaf=12, validation_fraction=0.15)

# AP

In [129]:
forest_bayes_search.best_score_

0.6878546912287747

# ROC auc

In [104]:
forest_bayes_search.best_score_

0.6895023895534506

In [79]:
aps = []
aucs = []
for train_index, test_index in skf.split(X, y):
    scaler = StandardScaler()
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    clf.fit(X_train, y_train)
    #knorae = KNORAE(pool_classifiers)
    #knorae.fit()
    log_probs = clf.predict_proba(X_test)[:, 1]
    ap = average_precision_score(y_test, log_probs)
    roc_auc = roc_auc_score(y_test, log_probs)
    aps.append(ap)
    aucs.append(roc_auc)

In [85]:
np.mean(aps)

0.14099473491817066

In [87]:
np.mean(aucs)

0.6449456398734204

# Knora

In [107]:
aps = []
aucs = []
for train_index, test_index in skf.split(X, y):
    scaler = StandardScaler()
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    X_train, X_dsel, y_train, y_dsel = train_test_split(X_train, y_train, test_size=0.5)
    
    clf = forest_bayes_search.best_estimator_
    clf.fit(X_train, y_train)
    knorae = KNORAE(clf)
    knorae.fit(X_dsel, y_dsel)
    
    log_probs = clf.predict_proba(X_test)[:, 1]
    ap = average_precision_score(y_test, log_probs)
    roc_auc = roc_auc_score(y_test, log_probs)
    aps.append(ap)
    aucs.append(roc_auc)

In [110]:
np.mean(aps)

0.1549412394889774

In [111]:
np.mean(aucs)

0.6263718833458845

## Complementary models

In [114]:
from dao import DrugAttritionOracle
import pickle
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from utils.metrics import table_metrics_trees
from sklearn.metrics import confusion_matrix
import shap

In [428]:
train = pd.read_csv('../data/processing_pipeline/TDC_predictions/train_subtasks_predictions.csv', index_col=0)
test = pd.read_csv('../data/processing_pipeline/TDC_predictions/test_subtasks_predictions.csv', index_col=0)
file = open('../production/complementary_model/random_forest_classifier.pkl', 'rb')
y_train = train['wd_consensus_1']
y_test = test['wd_consensus_1']
X_train = train.drop(columns=['chembl_id', 'standardized_smiles', 'wd_consensus_1'])
X_test = test.drop(columns=['chembl_id', 'standardized_smiles', 'wd_consensus_1'])
rf = pickle.load(file)
node_feat_importance = pd.DataFrame(data=rf.feature_importances_[np.newaxis], columns=X_train.columns, index=[0])
#feats = list(node_feat_importance.transpose().sort_values(0, ascending=False)[:13].index)

## Run on subset

# XGBoost

In [441]:
params = {
 'learning_rate' : [0.05,0.10,0.15,0.20,0.25,0.30],
 'max_depth' : [ 3, 4, 5, 6, 8, 10, 12, 15],
 'min_child_weight' : [ 1, 3, 5, 7 ],
 'gamma': [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 'colsample_bytree' : [ 0.3, 0.4, 0.5 , 0.7 ],
 'scale_pos_weight': [5, 10, 15, 20, 35],
}

In [442]:
import xgboost
classifier = xgboost.XGBClassifier()

In [443]:
rs_model=RandomizedSearchCV(clbassifier,param_distributions=params,n_iter=50,scoring='average_precision',n_jobs=-1,cv=5,verbose=3)

In [567]:
rs_model.best_params_

{'scale_pos_weight': 10,
 'min_child_weight': 5,
 'max_depth': 15,
 'learning_rate': 0.05,
 'gamma': 0.2,
 'colsample_bytree': 0.5}

In [580]:
classifier = xgboost.XGBClassifier(learning_rate=rs_model.best_params_['learning_rate'])

In [581]:
classifier.fit(X_train, y_train)

[15:59:12] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [584]:
classifier.get_booster().best_ntree_limit

100

In [444]:
rs_model.fit(X_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:  6.3min finished
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[12:43:15] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                   

In [562]:
X_test

,CYP2C9_Substrate_CarbonMangels,CYP2D6_Substrate_CarbonMangels,sr-are,CYP3A4_Veith,nr-er-lbd,nr-er,Solubility_AqSolDB,sr-atad5,Caco2_Wang,CYP2D6_Veith,...,CYP2C9_Veith,CYP1A2_Veith,HIA_Hou,nr-ppar-gamma,Clearance_Hepatocyte_AZ,Carcinogens_Languin,nr-aromatase,sr-mmp,sr-p53,predict_withdrawn
0,0.069458,0.004521,0.506577,0.224221,0.075100,0.000071,-2.688965,0.013394,-5.617864,0.001843,...,0.036047,0.006817,0.013520,0.008847,35.181927,0.286017,0.001043,2.742608e-06,0.009950,0.470054
1,0.103099,0.232109,0.827061,0.226972,0.403773,0.932831,-5.085590,0.917336,-4.535133,0.156842,...,0.782207,0.921282,0.998295,0.011342,75.808525,0.374289,0.004148,9.678469e-01,0.909958,0.606440
2,0.340530,0.142519,0.457970,0.958183,0.235134,0.000681,-3.813414,0.002442,-4.795193,0.151132,...,0.874266,0.762396,0.996635,0.769668,33.605370,0.310629,0.000067,1.533354e-01,0.012377,0.485826
3,0.025499,0.010939,0.081060,0.343454,0.095015,0.014898,-3.072849,0.001167,-3.959646,0.000006,...,0.322632,0.068863,0.991903,0.148467,113.047005,0.615662,0.000727,5.591495e-04,0.049619,0.129884
4,0.417281,0.264848,0.112804,0.030651,0.802562,0.937043,-5.873740,0.248247,-3.773222,0.007486,...,0.841268,0.853914,0.999501,0.001056,74.288340,0.583670,0.002604,1.606945e-01,0.145084,0.681620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,0.458754,0.308566,0.111702,0.000344,0.003962,0.715382,-0.882686,0.000556,-4.659957,0.000003,...,0.109648,0.207328,0.873437,0.005758,32.886288,0.352992,0.000028,6.340616e-07,0.007156,0.449080
391,0.167058,0.508774,0.665901,0.023806,0.548024,0.058295,-1.635835,0.725651,-5.748230,0.772372,...,0.330940,0.088069,0.997442,0.000385,69.781120,0.486183,0.000086,2.883503e-02,0.026883,0.631011
392,0.115259,0.038823,0.233657,0.000564,0.361210,0.060816,-3.895102,0.067972,-4.211524,0.011219,...,0.393423,0.027980,0.996202,0.000975,1.042210,0.252291,0.000168,1.127547e-03,0.001459,0.668740
393,0.234541,0.107649,0.050168,0.015203,0.033033,0.000702,-1.089897,0.025235,-4.674285,0.002292,...,0.365868,0.016981,0.859953,0.000113,8.636477,0.658230,0.000151,4.709835e-07,0.031638,0.381551


In [565]:
#optimal threshold F1 withdrawn class random forest
optimal_f1_score = []
optimal_threshold = []
for threshold in np.arange(0, 1, 0.01):
    predictions_df = train_pred_df.copy()
    predictions_df['predicted_class'] = 0
    predictions_df.loc[predictions_df['probabilities'] > threshold, 'predicted_class'] = 1
    optimal_f1_score.append(f1_score(
        predictions_df['target'], predictions_df['predicted_class'], average='binary'
    ))
    optimal_threshold.append(threshold)

optimal_f1_index = np.argmax(np.array(optimal_f1_score))
optimal_threshold = optimal_threshold[optimal_f1_index]

test_pred_df = pd.DataFrame({'probabilities': predictions[:, 1],
                            'wd_consensus_1': y_test,
                            
                             'predicted_class': rs_model.best_estimator_.predict(X_test)})
results = table_metrics_trees(test_pred_df, 'wd_consensus_1')

In [566]:
results

,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.397815,0.790555,0.832989,0.629392,0.5,0.292683,0.921833,0.966102,12,342,12,29


## Shap Graphs

In [102]:
from utils.metrics import table_metrics_trees
import shap

In [85]:
train = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/TDC_predictions/train_subtasks_predictions.csv', index_col=0)
test = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/TDC_predictions/test_subtasks_predictions.csv', index_col=0)

In [87]:
classifier = XGBClassifier(learning_rate=0.1,
        max_depth=3,
        min_child_weight=3,
        gamma=0.2,
        colsample_bytree=0.4,
        scale_pos_weight=10,
        n_estimators=200)

In [88]:
withdrawn_col = 'wd_consensus_1'

In [89]:
y_train = train[withdrawn_col]
y_test = test[withdrawn_col]
X_train = train.drop(columns=['chembl_id', 'standardized_smiles', withdrawn_col])
X_test = test.drop(columns=['chembl_id', 'standardized_smiles', withdrawn_col])

In [91]:
classifier.fit(X_train, y_train)

/home/dionizije/anaconda3/envs/withdrawn/lib/python3.6/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:38:20] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.4, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=3, missing=nan, monotone_constraints='()',
              n_estimators=200, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=10, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [186]:
top_5_features = list(
    pd.DataFrame(
        columns=X_train.columns,
        data=abs(np.mean(shap_values.values, axis=0))[np.newaxis]
    ).transpose().sort_values(0, ascending=False)[:5].index)

In [187]:
top_5_features

['ClinTox', 'predict_withdrawn', 'nr-er', 'CYP1A2_Veith', 'nr-ppar-gamma']

In [95]:
predictions = classifier.predict_proba(X_test, ntree_limit=classifier.get_booster().best_ntree_limit)
test_pred_df = pd.DataFrame({'probabilities': predictions[:, 1],
                             withdrawn_col: y_test,
                             'predicted_class': classifier.predict(X_test, ntree_limit=classifier.get_booster().best_ntree_limit)})

In [99]:
results = table_metrics_trees(test_pred_df, withdrawn_col)

In [100]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.488889,0.389249,0.794605,0.806394,0.730157,0.44898,0.536585,0.945087,0.923729,22,327,27,19


## Višnja ADME

In [412]:
train = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/TDC_predictions/train_subtasks_predictions.csv', index_col=0)
test = pd.read_csv('/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/TDC_predictions/test_subtasks_predictions.csv', index_col=0)

In [413]:
tox = pd.read_csv('/home/dionizije/Downloads/MasterDB_20Sep2021_ADMEPPredictor_Tox.csv', index_col=0)
adme_phys = pd.read_csv('/home/dionizije/Downloads/MasterDB_20Sep2021_ADMEPPredictor_PhysChem.csv', index_col=0)
rdkit = pd.read_csv("/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/descriptors/rdkit_descriptors.csv")
japtox = pd.read_csv("/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/descriptors/ADME-JapTox-RDKIT.csv").iloc[:, :34]
toxprint = pd.read_csv("/home/dionizije/Documents/drug_attrition_oracle/data/processing_pipeline/descriptors/toxprint_descriptors.csv")

In [414]:
toxprint_list = ["chain:alkaneBranch_neopentyl_C5",
    "bond:C(=O)N_carbamate_thio_generic",
    "chain:aromaticAlkene_Ph-C4_phenylbutadiene",
    "chain:aromaticAlkane_Ph-C1_acyclic_generic",
    "ring:aromatic_benzene",
    "bond:COH_alcohol_aliphatic_generic",
    "bond:COH_alcohol_sec-alkyl",
    "ring:aromatic_phenyl",
    "bond:C(=O)N_carboxamide_(NHR)",
    "group:aminoAcid_aminoAcid_generic",
    "chain:aromaticAlkane_Ph-C1-Ph",
    "bond:C=O_carbonyl_ab-unsaturated_generic",
    "chain:alkaneCyclic_ethyl_C2_(connect_noZ)",
    "bond:CC(=O)C_ketone_aliphatic_acyclic",
    "chain:aromaticAlkane_Ar-C-Ar",
    "bond:CC(=O)C_ketone_generic",
    "chembl_id"]
toxprint = toxprint[toxprint_list]

In [415]:
toxprint

,chain:alkaneBranch_neopentyl_C5,bond:C(=O)N_carbamate_thio_generic,chain:aromaticAlkene_Ph-C4_phenylbutadiene,chain:aromaticAlkane_Ph-C1_acyclic_generic,ring:aromatic_benzene,bond:COH_alcohol_aliphatic_generic,bond:COH_alcohol_sec-alkyl,ring:aromatic_phenyl,bond:C(=O)N_carboxamide_(NHR),group:aminoAcid_aminoAcid_generic,chain:aromaticAlkane_Ph-C1-Ph,bond:C=O_carbonyl_ab-unsaturated_generic,chain:alkaneCyclic_ethyl_C2_(connect_noZ),bond:CC(=O)C_ketone_aliphatic_acyclic,chain:aromaticAlkane_Ar-C-Ar,bond:CC(=O)C_ketone_generic,chembl_id
0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,CHEMBL1091250
1,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,CHEMBL1601
2,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,CHEMBL2110774
3,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,CHEMBL385517
4,0,0,0,1,1,0,0,1,1,1,0,0,0,0,0,0,CHEMBL1201779
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,0,0,0,1,1,0,0,1,1,0,0,0,0,0,0,0,CHEMBL918
2499,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,CHEMBL926
2500,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,1,CHEMBL370252
2501,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,CHEMBL1201336


In [398]:
japtox = japtox.dropna()

In [344]:
tox = tox.drop(columns=['test_train', 'drugbank_id', 'mw_ap', 'drugbank_three_class', 'sum', 'wd_consensus_1', 'wd_consensus_2', 'wd_consensus_3',
                       'wd_consensus_1_NO_DISCONT', 'wd_consensus_2_NO_DISCONT', 'wd_consensus_3_NO_DISCONT'])
adme_phys = adme_phys.drop(columns=['test_train', 'mw_ap', 'drugbank_three_class', 'sum', 'wd_consensus_1', 'wd_consensus_2', 'wd_consensus_3',
                       'wd_consensus_1_NO_DISCONT', 'wd_consensus_2_NO_DISCONT', 'wd_consensus_3_NO_DISCONT'])

In [338]:
train = train[['predict_withdrawn', 'chembl_id', 'wd_consensus_1']]
test = test[['predict_withdrawn', 'chembl_id', 'wd_consensus_1']]

In [345]:
train = train.merge(tox, how='inner', on='chembl_id')
train = train.merge(adme_phys, how='inner', on='chembl_id')
train = train.drop_duplicates('chembl_id')

In [363]:
train = train.merge(rdkit, how='inner', on='chembl_id')
test = test.merge(rdkit, how='inner', on='chembl_id')
train = train.drop_duplicates('chembl_id')
test = test.drop_duplicates('chembl_id')

In [346]:
test = test.merge(tox, how='inner', on='chembl_id')
test = test.merge(adme_phys, how='inner', on='chembl_id')
test = test.drop_duplicates('chembl_id')

In [399]:
train = train.merge(japtox, how='inner', on='chembl_id')
test = test.merge(japtox, how='inner', on='chembl_id')
train = train.drop_duplicates('chembl_id')
test = test.drop_duplicates('chembl_id')

In [416]:
train = train.merge(toxprint, how='inner', on='chembl_id')
test = test.merge(toxprint, how='inner', on='chembl_id')
train = train.drop_duplicates('chembl_id')
test = test.drop_duplicates('chembl_id')

In [418]:
y_train = train['wd_consensus_1']
y_test = test['wd_consensus_1']

X_train = train.drop(columns=['wd_consensus_1', 'chembl_id', 'standardized_smiles'])
X_test = test.drop(columns=['wd_consensus_1', 'chembl_id', 'standardized_smiles'])

In [419]:
X_train.columns

Index(['CYP2C9_Substrate_CarbonMangels', 'CYP2D6_Substrate_CarbonMangels',
       'sr-are', 'CYP3A4_Veith', 'nr-er-lbd', 'nr-er', 'Solubility_AqSolDB',
       'sr-atad5', 'Caco2_Wang', 'CYP2D6_Veith', 'Skin Reaction', 'PPBR_AZ',
       'Pgp_Broccatelli', 'BBB_Martins', 'nr-ar-lbd', 'VDss_Lombardo',
       'CYP3A4_Substrate_CarbonMangels', 'Lipophilicity_AstraZeneca',
       'LD50_Zhu', 'hERG', 'Bioavailability_Ma', 'nr-ahr', 'DILI', 'nr-ar',
       'AMES', 'CYP2C19_Veith', 'ClinTox', 'Half_Life_Obach', 'sr-hse',
       'CYP2C9_Veith', 'CYP1A2_Veith', 'HIA_Hou', 'nr-ppar-gamma',
       'Clearance_Hepatocyte_AZ', 'Carcinogens_Languin', 'nr-aromatase',
       'sr-mmp', 'sr-p53', 'predict_withdrawn',
       'chain:alkaneBranch_neopentyl_C5', 'bond:C(=O)N_carbamate_thio_generic',
       'chain:aromaticAlkene_Ph-C4_phenylbutadiene',
       'chain:aromaticAlkane_Ph-C1_acyclic_generic', 'ring:aromatic_benzene',
       'bond:COH_alcohol_aliphatic_generic', 'bond:COH_alcohol_sec-alkyl',
       '

In [420]:
    params = {
        'learning_rate': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
        'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
        'min_child_weight': [1, 3, 5, 7],
        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
        'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
        'scale_pos_weight': [5, 10, 15, 20, 35],
        'n_estimators': [100, 200, 300, 400, 500],
    }

In [421]:
classifier = XGBClassifier()

In [423]:
rs_model = RandomizedSearchCV(classifier, param_distributions=params, n_iter=100, scoring='average_precision',
                              n_jobs=-1, cv=6, verbose=3)
rs_model.fit(X_train, y_train)

Fitting 6 folds for each of 100 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done 496 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 28.1min finished
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[11:16:59] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=6,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           validate_parameters=None,
                                           verbosity=None),
                   n_iter=100, n_jobs=-1,
          

In [424]:
predictions = rs_model.best_estimator_.predict_proba(X_test)
test_pred_df = pd.DataFrame({'probabilities': predictions[:, 1],
                             withdrawn_col: y_test,
                             'predicted_class': rs_model.predict(X_test)})

In [425]:
results = table_metrics_trees(test_pred_df, withdrawn_col)

* Predicted + toxprint

In [426]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.417266,0.43451,0.785302,0.833953,0.756201,0.295918,0.707317,0.959596,0.805085,29,285,69,12


* Predicted + rdkit

In [408]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.470588,0.365484,0.79933,0.789996,0.710004,0.454545,0.487805,0.940171,0.932203,20,330,24,21


* Predicted

In [387]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.453782,0.381002,0.793424,0.818038,0.757234,0.346154,0.658537,0.955836,0.855932,27,303,51,14


* RDKIT + predicted

In [371]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.41791,0.467745,0.784435,0.828166,0.653783,0.538462,0.341463,0.926829,0.966102,14,342,12,27


* adme + predicted

In [356]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.414414,0.403796,0.789654,0.834711,0.714104,0.328571,0.560976,0.944615,0.867232,23,307,47,18


* tox only

In [319]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.266667,0.349915,0.798773,0.808185,0.582024,0.421053,0.195122,0.912234,0.968927,8,343,11,33


* Results tox + physchem

In [303]:
results

,F1 score,AP withdrawn,AP approved,AUROC withdrawn,Balanced accuracy,Precision withdrawn,Recall withdrawn,Precision approved,Recall approved,True positives,True negatives,False positives,False negatives
0,0.384,0.369677,0.793242,0.834849,0.707937,0.285714,0.585366,0.945338,0.830508,24,294,60,17


# WD Models with ATC codes

In [210]:
from src.utils.descriptors_list import toxprint_descriptors_10pct
import re

In [181]:
root = Path('/home/dionizije/Documents/drug_attrition_oracle')

In [206]:
train = pd.read_csv(root / 'data/processing_pipeline/train/train.csv')[['chembl_id', withdrawn_col]]
train = train.sample(frac=1, random_state=0)  # shuffle
test = pd.read_csv(root / 'data/processing_pipeline/test/test.csv')[['chembl_id', withdrawn_col]]
toxprints = pd.read_csv(root / 'data/processing_pipeline/descriptors/toxprint_descriptors.csv')
chembl_ids = toxprints['chembl_id']
toxprints = toxprints[toxprint_descriptors_10pct] # drop mostly 0 zescriptors
toxprints['chembl_id'] = chembl_ids
master_atc = pd.read_csv(root / 'data/processing_pipeline/master_atc.csv', index_col=0)

In [211]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
toxprints.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in
          toxprints.columns.values]

In [212]:
train = train.merge(master_atc, how='inner', on='chembl_id')
test = test.merge(master_atc, how='inner', on='chembl_id')

train = train.merge(toxprints, how='inner', on='chembl_id')
test = test.merge(toxprints, how='inner', on='chembl_id')

In [213]:
    train['atc_code'] = train['atc_code'].str.split('0').str[0]
    train['atc_code'] = train['atc_code'].str.split('1').str[0]

    test['atc_code'] = test['atc_code'].str.split('0').str[0]
    test['atc_code'] = test['atc_code'].str.split('1').str[0]

In [214]:
params = {
    'learning_rate': [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    'max_depth': [3, 4, 5, 6, 8, 10, 12, 15],
    'min_child_weight': [1, 3, 5, 7],
    'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
    'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
    'scale_pos_weight': [5, 10, 15, 20, 35],
    'n_estimators': [1],
}

In [288]:
ap_overall = []
atc_codes = []
auroc_overall = []
results = []
for i in list(train['atc_code'].unique()):
    train_subset = train.loc[train['atc_code'] == i]
    test_subset = test.loc[test['atc_code'] == i]

    cv_splitter = StratifiedKFold(
    n_splits=6,
    shuffle=True,
    random_state=0)
    
    features_across_fold = []
    ap_fold = []
    auroc_fold = []
    for k, (train_index, test_index) in enumerate(
            cv_splitter.split(train_subset, train_subset[withdrawn_col])
    ):
        y_train = train_subset.iloc[train_index]['wd_consensus_1']
        y_test = train_subset.iloc[test_index]['wd_consensus_1']
        X_train = train_subset.iloc[train_index].drop(columns=['wd_consensus_1', 'chembl_id', 'atc_code'])
        X_test = train_subset.iloc[test_index].drop(columns=['wd_consensus_1', 'chembl_id', 'atc_code'])

        classifier = XGBClassifier()
        rs_model = RandomizedSearchCV(classifier, param_distributions=params,
                                      n_iter=1, scoring='average_precision',
                                      n_jobs=-1, cv=6, verbose=3)
        rs_model.fit(X_train, y_train)
        predict_proba = rs_model.best_estimator_.predict_proba(X_test)
        ap = average_precision_score(y_test, predict_proba[:, 1])
        auroc = roc_auc_score(y_test, predict_proba[:, 1])
        sorted_idx = rs_model.best_estimator_.feature_importances_.argsort()
        sorted_importances = list(X_train.columns[sorted_idx][-10:])
        features_across_fold.append(sorted_importances)
        ap_fold.append(ap)
        auroc_fold.append(auroc)
    top_features = (pd.DataFrame(features_across_fold).melt().groupby('value').sum().sort_values('variable', ascending=False)[:10].index)
    results.append(top_features)
    atc_codes.append(i)
    ap_overall.append(np.mean(ap_fold))
    auroc_overall.append(np.mean(auroc_fold))

Fitting 6 folds for each of 1 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.5s finished
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


[13:59:23] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 6 folds for each of 1 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.3s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.5s finished
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].


[13:59:24] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 6 folds for each of 1 candidates, totalling 6 fits


...

[14:00:03] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Fitting 6 folds for each of 1 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    0.3s remaining:    0.3s


[14:00:04] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    0.6s finished
The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
invalid value encountered in true_divide


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [294]:
    results = []
    atc_codes = []
    columns = []
    for i in list(train['atc_code'].unique()):
        #try:
            train_subset = train.loc[train['atc_code'] == i]
            test_subset = test.loc[test['atc_code'] == i]

            y_train = train_subset['wd_consensus_1']
            y_test = test_subset['wd_consensus_1']
            X_train = train_subset.drop(columns=['wd_consensus_1', 'chembl_id', 'standardized_smiles', 'atc_code'])
            X_test = test_subset.drop(columns=['wd_consensus_1', 'chembl_id', 'standardized_smiles', 'atc_code'])

            classifier = XGBClassifier()
            rs_model = RandomizedSearchCV(classifier, param_distributions=params,
                                          n_iter=100, scoring='average_precision',
                                          n_jobs=-1, cv=6, verbose=3)
            rs_model.fit(X_train, y_train)

            predictions = rs_model.best_estimator_.predict_proba(X_test)
            test_pred_df = pd.DataFrame({'probabilities': predictions[:, 1],
                                         withdrawn_col: y_test,
                                         'predicted_class': rs_model.predict(X_test)})
            results.append(table_metrics_trees(test_pred_df, withdrawn_col).values[0])
            columns.append(table_metrics_trees(test_pred_df, withdrawn_col).columns)
            atc_codes.append(i)
        #except:
            #continue

    results_df = pd.DataFrame(results, columns=columns[0], index=atc_codes)

KeyError: "['standardized_smiles'] not found in axis"

In [289]:
results_df = pd.DataFrame(results, index=atc_codes)

In [290]:
results_df['mean_average_precision'] = ap_overall

In [291]:
results_df['mean_auroc'] = auroc_overall

In [292]:
results_df

,0,1,2,3,4,5,6,7,8,9,mean_average_precision,mean_auroc
J,ring:hetero__5__Z_1_3-Z,bond:CC(=O)C_ketone_generic,bond:C(=O)N_carboxamide_generic,chain:aromaticAlkane_Ph-C1_acyclic_generic,chain:alkaneLinear_ethyl_C2(H_gt_1),bond:COC_ether_aliphatic,bond:COC_ether_aliphatic__aromatic,bond:COH_alcohol_aliphatic_generic,bond:CN_amine_sec-NH_generic,ring:hetero__6_6__Z_generic,0.158193,0.507341
A,bond:COH_alcohol_aliphatic_generic,bond:C(=O)N_carboxamide_generic,chain:alkaneLinear_ethyl_C2(H_gt_1),bond:COH_alcohol_sec-alkyl,ring:aromatic_benzene,chain:aromaticAlkane_Ph-C1_acyclic_generic,bond:CN_amine_ter-N_aliphatic,bond:CC(=O)C_ketone_aliphatic_generic,chain:alkaneCyclic_hexyl_C6,bond:COC_ether_aliphatic,0.216338,0.647837
G,ring:hetero__6__Z_generic,chain:alkaneBranch_isopropyl_C3,bond:C(=O)N_carboxamide_generic,chain:alkaneLinear_ethyl_C2(H_gt_1),bond:COH_alcohol_aromatic,bond:CX_halide_aromatic-X_generic,chain:alkaneBranch_neopentyl_C5,ring:hetero__5__Z_1_3-Z,ring:hetero__6_6__Z_generic,chain:alkaneLinear_propyl_C3,0.242896,0.530556
S,chain:alkaneBranch_neopentyl_C5,bond:CX_halide_aromatic-X_generic,ring:hetero__6__Z_generic,bond:C(=O)N_carboxamide_generic,chain:alkaneLinear_ethyl_C2(H_gt_1),bond:CC(=O)C_ketone_aliphatic_generic,bond:COC_ether_aliphatic,bond:CN_amine_sec-NH_alkyl,chain:aromaticAlkane_Ph-C1_acyclic_generic,bond:COH_alcohol_sec-alkyl,0.248083,0.743365
N,chain:alkaneLinear_propyl_C3,chain:alkaneBranch_neopentyl_C5,chain:alkaneLinear_ethyl_C2_(connect_noZ_CN=4),chain:alkaneBranch_isopropyl_C3,bond:CX_halide_aromatic-X_generic,bond:C(=O)N_carboxamide_generic,bond:CN_amine_sec-NH_alkyl,bond:CN_amine_alicyclic_generic,ring:hetero__6__N_pyridine_generic,ring:hetero__5__Z_1_3-Z,0.195870,0.594190
C,ring:aromatic_benzene,bond:COH_alcohol_aliphatic_generic,chain:alkaneBranch_isopropyl_C3,chain:alkaneLinear_propyl_C3,bond:CC(=O)C_ketone_generic,bond:CX_halide_aromatic-X_generic,bond:CN_amine_pri-NH2_generic,chain:aromaticAlkane_Ph-C1_cyclic,ring:hetero__6__N_pyridine_generic,bond:CC(=O)C_ketone_aliphatic_generic,0.233856,0.655864
R,bond:CC(=O)C_ketone_generic,bond:COH_alcohol_aliphatic_generic,bond:CN_amine_sec-NH_alkyl,chain:alkaneBranch_isopropyl_C3,ring:hetero__5__Z_1_3-Z,chain:alkaneLinear_ethyl_C2(H_gt_1),bond:CX_halide_aromatic-X_generic,chain:alkaneLinear_ethyl_C2_(connect_noZ_CN=4),ring:aromatic_phenyl,chain:aromaticAlkane_Ph-C1_acyclic_generic,0.207821,0.707437
M,bond:C(=O)N_carboxamide_generic,chain:alkaneLinear_ethyl_C2_(connect_noZ_CN=4),bond:COH_alcohol_aliphatic_generic,chain:aromaticAlkane_Ph-C1_acyclic_generic,bond:C(=O)N_carboxamide_(NHR),ring:aromatic_benzene,bond:CN_amine_sec-NH_generic,bond:CX_halide_aromatic-X_generic,ring:aromatic_phenyl,bond:C(=O)O_carboxylicAcid_alkyl,0.668128,0.793803
L,chain:alkaneBranch_neopentyl_C5,ring:hetero__6__Z_generic,chain:alkaneLinear_propyl_C3,bond:COH_alcohol_aliphatic_generic,chain:alkaneLinear_ethyl_C2(H_gt_1),ring:hetero__6_6__Z_generic,chain:aromaticAlkane_Ph-C1_acyclic_generic,bond:C(=O)N_carboxamide_generic,bond:COH_alcohol_aromatic,bond:COC_ether_aliphatic,0.123778,0.584656
D,chain:alkaneCyclic_hexyl_C6,bond:COH_alcohol_aromatic,bond:C(=O)N_carboxamide_generic,bond:CX_halide_aromatic-X_generic,ring:hetero__6__Z_generic,ring:hetero__5__Z_1_3-Z,bond:CC(=O)C_ketone_aliphatic_generic,bond:C(=O)O_carboxylicAcid_generic,ring:aromatic_phenyl,chain:alkaneLinear_ethyl_C2(H_gt_1),0.256457,0.688779
